In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
import psutil
import os
directory = '/scratch/6256481/'
from glob import glob
import numpy as np
os.chdir(directory + 'python/py_code')
from dem import find_dem_bounds_FAB, find_dem_FAB
from support import check_memory

from osgeo import gdal
from dem import get_raster_vrt
vrt_file = directory + "input_created/FAB_dem_vrt.vrt"
vrt_ds = gdal.Open(vrt_file)

/home/6256481/.conda/envs/RivAlg10/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [2]:
dfDemBounds = find_dem_bounds_FAB(directory, 'EPSG:4326')

In [3]:
df = gpd.read_file(np.sort(glob(directory + 'results/new_segments/vector/*'))[0])

In [4]:
import sys
import psutil
import gc
def list_variables():
    """List all variables in memory with their sizes."""
    variables = {name: sys.getsizeof(value) for name, value in globals().items()}
    return sorted(variables.items(), key=lambda x: x[1], reverse=True)  # Sort by size


def list_objects():
    """Get a list of all objects in memory."""
    return [str(type(obj)) for obj in gc.get_objects()]  # Limit to 20 items

def get_object_ids():
    """Get a set of object IDs currently in memory."""
    return {id(obj): obj for obj in gc.get_objects()}
 # Show first 20 objects in memory

def list_open_files():
    """List all currently open files by the Python process."""
    process = psutil.Process()
    return [f.path for f in process.open_files()]

def find_largest_objects(objects, n=10):
    """Find the largest objects currently tracked by the garbage collector."""
    # objects = gc.get_objects()
    sized_objects = [(sys.getsizeof(obj), type(obj), obj) for obj in objects]
    sized_objects.sort(reverse=True, key=lambda x: x[0])
    
    print(f"\nTop {n} largest objects:")
    for size, obj_type, obj in sized_objects[:n]:
        print(f"{obj_type}: {size} bytes")

    return sized_objects[:n]  # Return for debugging if needed


def find_and_remove_largest_objects(objects, n=10):
    """Find and attempt to remove the largest objects tracked by the garbage collector."""
    # objects = gc.get_objects()
    sized_objects = [(sys.getsizeof(obj), type(obj), obj) for obj in objects]
    sized_objects.sort(reverse=True, key=lambda x: x[0])  # Sort by size (largest first)

    print(f"\nTop {n} largest objects before cleanup:")
    for size, obj_type, obj in sized_objects[:n]:
        print(f"{obj_type}: {size} bytes")

    # Attempt to remove these objects
    for _, _, obj in sized_objects[:n]:
        if isinstance(obj, (list, dict, set, tuple)):  # Handle common container types
            obj.clear()
        elif isinstance(obj, (str, bytes, bytearray)):
            obj = ""  # Replace large strings with an empty string
        else:
            obj = None  # Try removing references

    gc.collect()  # Force garbage collection

    print(f"Garbage collected. Remaining objects: {len(gc.get_objects())}")

In [5]:
A = check_memory()
gc.collect()

for i, r in df.iloc[0:10].iterrows():
    before_objects = get_object_ids()
    print(check_memory())
    dfR = df[df.index == i].copy()
    dfR = dfR.to_crs(r['localCRS'])

    R =  get_raster_vrt(vrt_ds, dfR,60000, r['localCRS'], 'EPSG:4326')


    dfR = None
    del dfR
    gc.collect()
    print(len(list_objects())) 
    after_objects = get_object_ids()

    new_object_ids = set(after_objects.keys()) - set(before_objects.keys())
    new_objects    = [after_objects[obj_id] for obj_id in new_object_ids]
    # find_and_remove_largest_objects(new_objects)
    print(check_memory())
    print()




274.85
180817
457.95

463.36
180884
525.39

530.8
180951
693.37

698.8
181018
767.13

772.77
181085
807.01

812.4
181152
891.97

897.33
181256
989.14

994.94
181323
1083.51

1089.04
181397
1175.46

1181.03
181464
1270.37



In [ ]:
221352

221456

221560

221664

221768

221872

221976

222087

222191

222295

In [6]:
A1 = find_largest_objects(before_objects)


Top 10 largest objects:
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes


In [7]:
A2 = find_largest_objects(after_objects)
A2


Top 10 largest objects:
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes
<class 'int'>: 32 bytes


[(32, int, 140621036950624),
 (32, int, 140621068337408),
 (32, int, 140621068338304),
 (32, int, 140621068353024),
 (32, int, 140621068345472),
 (32, int, 140621071908928),
 (32, int, 140621071917504),
 (32, int, 140621068031296),
 (32, int, 140621068538624),
 (32, int, 140621068536224)]

In [8]:
A = find_largest_objects(new_objects, n = 15)


Top 15 largest objects:
<class 'dict'>: 10485848 bytes
<class 'pandas.core.indexes.base.Index'>: 33136 bytes
<class 'pandas.core.indexes.base.Index'>: 33136 bytes
<class 'pandas.core.indexes.base.Index'>: 32304 bytes
<class 'pandas.core.indexes.base.Index'>: 32304 bytes
<class 'frozenset'>: 728 bytes
<class 'collections.defaultdict'>: 232 bytes
<class 'dict'>: 184 bytes
<class 'dict'>: 184 bytes
<class '_io.StringIO'>: 184 bytes
<class 'function'>: 160 bytes
<class 'xarray.core.dataarray.DataArray'>: 104 bytes
<class 'xarray.core.variable.IndexVariable'>: 88 bytes
<class 'list'>: 88 bytes
<class 'list'>: 88 bytes


In [20]:
A[11]

(104,
 xarray.core.dataarray.DataArray,
 <xarray.DataArray (y: 4138, x: 4034)> Size: 67MB
 array([[-9999.  ,    30.54,    30.55, ..., -9999.  , -9999.  , -9999.  ],
        [-9999.  ,    30.53,    30.54, ..., -9999.  , -9999.  , -9999.  ],
        [-9999.  ,    30.51,    30.52, ...,    31.58,    31.62,    31.64],
        ...,
        [   17.75,    17.69,    18.11, ..., -9999.  , -9999.  , -9999.  ],
        [   17.56,    17.57,    17.62, ..., -9999.  , -9999.  , -9999.  ],
        [   17.35,    17.38,    17.56, ..., -9999.  , -9999.  , -9999.  ]],
       dtype=float32)
 Coordinates:
   * y        (y) float64 33kB 9.208e+04 9.205e+04 ... -3.545e+04 -3.548e+04
   * x        (x) float64 32kB 1.886e+05 1.886e+05 ... 3.129e+05 3.129e+05)

In [19]:
A[13]

(88,
 list,
 [<weakref at 0x7fe4e479fbf0; dead>,
  <weakref at 0x7fe4e44f8ae0; dead>,
  <weakref at 0x7fe4e44f89f0; to 'Index' at 0x7fe4e4d6b1a0>])

In [21]:
refs = gc.get_referrers(A[11])
print(f"\nFound {len(refs)} references to object.")


Found 5 references to object.


In [22]:
for ref in refs:
    print(type(ref), ref)  # Print some context about the references


<class 'dict'> 

KeyboardInterrupt: 

In [ ]:
# Find the dictionary in gc.get_objects()
objects = gc.get_objects()
for obj in objects:
    if isinstance(obj, dict) and 'y' in obj and 'x' in obj:
        print("Target dictionary found!")
        find_object_references(obj)
        break  # Stop after finding the first match

['/home/6256481/.ipython/profile_default/history.sqlite', '/home/6256481/.ipython/profile_default/history.sqlite', '/home/6256481/.conda/envs/RivAlg10/share/proj/proj.db']


/home/6256481/.conda/envs/RivAlg10/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [31]:


for i, r in df.iloc[0:10].iterrows():
    
    dfR = df[df.index == i].copy()
    dfR = dfR.to_crs(r['localCRS'])
    print(check_memory())
    R = get_raster_vrt(vrt_ds, dfR,60000, r['localCRS'], 'EPSG:4326')
    print(check_memory())
    print()

5149.57
5212.83

5212.83
5212.64

5212.64
5177.05

5177.05
5193.92

5193.92
5225.99

5225.99
5223.32

5223.32
5231.55

5231.55
5228.42

5228.42
5176.58

5176.58
5179.16



In [9]:
import gc

# test netcdf

In [ ]:
import xarray as xr
import geopandas as gpd
from shapely.wkt import loads

# Load NetCDF file
ds = xr.open_dataset("output_with_wkt.nc")

# Convert back to DataFrame
df = ds.to_dataframe().reset_index()

# Convert WKT strings back to geometries
df["geometry"] = df["wkt_geometry"].apply(loads)

# Convert back to GeoDataFrame
gdf_restored = gpd.GeoDataFrame(df, geometry="geometry")

print(gdf_restored.head())  # Verify restored geometries